# Leverage a golden_standard to create a dupe dataset

In [49]:
import os
import pandas as pd

Address_df = pd.read_csv('Data_source/Sample_15_test/sample_data/Address_s.csv')  
ContactPoint_df = pd.read_csv('Data_source/Sample_15_test/sample_data/ContactPoint_s.csv')
HealthcareOrganization_df = pd.read_csv('Data_source/Sample_15_test/sample_data/HealthcareOrganization_s.csv')
HealthcarePersonnel_df = pd.read_csv('Data_source/Sample_15_test/sample_data/HealthcarePersonnel_s.csv')
Person_df = pd.read_csv('Data_source/Sample_15_test/sample_data/Person_s.csv')
ServiceDepartment_df = pd.read_csv('Data_source/Sample_15_test/sample_data/ServiceDepartment_s.csv')


# Address_df = pd.read_csv('Data_source/Sample_35_train/train_data/Address.csv')  
# ContactPoint_df = pd.read_csv('Data_source/Sample_35_train/train_data/ContactPoint.csv')
# HealthcareOrganization_df = pd.read_csv('Data_source/Sample_35_train/train_data/HealthcareOrganization.csv')
# HealthcarePersonnel_df = pd.read_csv('Data_source/Sample_35_train/train_data/HealthcarePersonnel.csv')
# Person_df = pd.read_csv('Data_source/Sample_35_train/train_data/Person.csv')
# ServiceDepartment_df = pd.read_csv('Data_source/Sample_35_train/train_data/ServiceDepartment.csv')

# Specify the encoding to handle the decoding error
golden_record = pd.read_csv('test_golden_standard_high.csv', encoding='latin1')

entity_dataframes = {
    'Address': Address_df,
    'ContactPoint': ContactPoint_df,
    'HealthcareOrganization': HealthcareOrganization_df,
    'HealthcarePersonnel': HealthcarePersonnel_df,
    'Person': Person_df,
    'ServiceDepartment': ServiceDepartment_df
}

In [50]:
golden_record

,original_id,duplicate_id,entity_type,variation_type,field_name,original_value,varied_value
0,b2b5591a-9d7f-442b-85e2-b0787a3e9905,d46bbbcf-2fd2-5e4a-9850-8aee8e375ea0,Address,city_typo,city,Kattendijke,Kattedijke
1,d693bf11-7721-4931-9734-2123e3ed9637,c64f325c-12a5-5268-a7b3-d5cc573c5e4a,Address,city_typo,city,Deursen-Dennenburg,Deursen-Dennennburg
2,fbfa7549-13ee-4ee8-950e-18400e60f583,643e92eb-f928-50f3-8d08-ddde2bc715bb,Address,city_typo,city,IJsselstein,IJsselstdein
3,1c7bc6c8-d62f-4e8c-abca-09663eef04eb,185ed603-33f5-5e5c-adf5-37bdd9e8409c,Address,postal_format,postalCode,83054,83054
4,406ec398-9a4a-4a9c-a735-a17352ec0382,fe1bfcc2-9508-5bf6-965f-c299893ce69d,Address,house_number_suffix,text,ChloÃ©laan 150,ChloÃ©laan 150A
...,...,...,...,...,...,...,...
933,74286356-3c91-4458-a4b2-7e8301cbd9c2,5dbdda62-19cb-51a1-b877-154b65c19a9a,ContactPoint,email_typo,email,Robinson.Obstetric@dept.healthcare.org,Robinson.Obstehtric@dept.healthcare.org
934,047595fb-338b-4372-a00c-ea03b7d52dce,fc710d84-5b21-527e-9710-657efce3bf63,ContactPoint,email_domain_change,email,Peck.Neurologic@dept.healthcare.org,Peck.Neurologic@dept.healthcare.nl
935,e16e3336-77b1-4e21-bccc-c53ac36a4162,86b8120b-46a5-5ce6-81dc-ae91b412b000,ContactPoint,email_typo,email,Mcneil.Plastic@dept.healthcare.org,Mcneil.Plasttic@dept.healthcare.org
936,9177b6d4-5f2e-4167-b13e-ceb14ad12b4c,a64c9fd3-ed21-53dd-9e91-cf16e0f5aa19,ContactPoint,email_domain_change,email,FigueroaStrong@healthcare.org,FigueroaStrong@healthcare.nl


adding anchor to make sure we can backtrack the original identifiers iff anchor does not exist already

In [ ]:
# Rename 'identifier' column to 'anchor' in all entity dataframes


for df in entity_dataframes.values():
    if 'identifier' in df.columns and 'anchor' not in df.columns:
        df['anchor'] = df['identifier']

        

Create new identifiers for the whole dataset

In [53]:
from faker import Faker

fake = Faker()

# Create a copy to avoid modifying original data
A_copy = entity_dataframes['Address'].copy()
CP_copy = entity_dataframes['ContactPoint'].copy()
P_copy =  entity_dataframes['Person'].copy()
HO_copy = entity_dataframes['HealthcareOrganization'].copy()
SD_copy = entity_dataframes['ServiceDepartment'].copy()
HP_copy = entity_dataframes['HealthcarePersonnel'].copy()

# Save old values
old_valuesHO = {
    'identifier': HO_copy['identifier'].copy(),
    'address': HO_copy['address'].copy(),
    'contactPoint': HO_copy['contactPoint'].copy()
}

# Generate new UUIDs for each field

HO_copy['identifier'] = HO_copy['identifier'].apply(lambda x: fake.uuid4())
HO_copy['address'] = HO_copy['address'].apply(lambda x: fake.uuid4())
HO_copy['contactPoint'] = HO_copy['contactPoint'].apply(lambda x: fake.uuid4())

# Create copies of dataframes
old_valuesSD = {
    'identifier': SD_copy['identifier'].copy(),
    'address': SD_copy['address'].copy(),
    'contactPoint': SD_copy['contactPoint'].copy()
}

# Generate new UUIDs for each field in ServiceDepartment_df
SD_copy['identifier'] = SD_copy['identifier'].apply(lambda x: fake.uuid4())
SD_copy['address'] = SD_copy['address'].apply(lambda x: fake.uuid4())
SD_copy['contactPoint'] = SD_copy['contactPoint'].apply(lambda x: fake.uuid4())

old_valuesHP = {
    'identifier': HP_copy['identifier'].copy()
}
# Generate new UUIDs for Healthcare Personnel identifiers
HP_copy['identifier'] = HP_copy['identifier'].apply(lambda x: fake.uuid4())

In [54]:
# Create a mapping of old to new UUIDs
HO_uuid_mapping = {
    old_valuesHO['identifier'].iloc[i]: HO_copy['identifier'].iloc[i]
    for i in range(len(HO_copy))
}

# Update references in other dataframes (using the copies)
# Update institution references in Healthcare_Personnel_df copy
HP_copy['institution'] = HP_copy['institution'].map(HO_uuid_mapping).fillna(HP_copy['institution'])

# Update isPartOf references in ServiceDepartment_df copy
SD_copy['isPartOf'] = SD_copy['isPartOf'].map(HO_uuid_mapping).fillna(SD_copy['isPartOf'])

# Update contact point references
ho_contact_mapping = {
    old_valuesHO['contactPoint'].iloc[i]: HO_copy['contactPoint'].iloc[i]
    for i in range(len(HO_copy))
}

# Update identifier in ContactPoint_df copy where it matches old contact points
CP_copy['identifier'] = CP_copy['identifier'].map(ho_contact_mapping).fillna(CP_copy['identifier'])

# Update address references
ho_address_mapping = {
    old_valuesHO['address'].iloc[i]: HO_copy['address'].iloc[i]
    for i in range(len(HO_copy))
}

# Update identifier in Address_df copy where it matches old addresses
A_copy['identifier'] = A_copy['identifier'].map(ho_address_mapping).fillna(A_copy['identifier'])

# Create mapping for ServiceDepartment
sd_uuid_mapping = {
    old_valuesSD['identifier'].iloc[i]: SD_copy['identifier'].iloc[i]
    for i in range(len(SD_copy))
}

# Create address mapping for ServiceDepartment
sd_address_mapping = {
    old_valuesSD['address'].iloc[i]: SD_copy['address'].iloc[i]
    for i in range(len(SD_copy))
}

# Create contact point mapping for ServiceDepartment
sd_contact_mapping = {
    old_valuesSD['contactPoint'].iloc[i]: SD_copy['contactPoint'].iloc[i]
    for i in range(len(SD_copy))
}

# Update references in Address_df copy for ServiceDepartment
A_copy['identifier'] = A_copy['identifier'].map(sd_address_mapping).fillna(A_copy['identifier'])

# Update references in ContactPoint_df copy for ServiceDepartment
CP_copy['identifier'] = CP_copy['identifier'].map(sd_contact_mapping).fillna(CP_copy['identifier'])

# Create mapping for Personnel
hp_uuid_mapping = {
    old_valuesHP['identifier'].iloc[i]: HP_copy['identifier'].iloc[i]
    for i in range(len(HP_copy))
}

# Update references in Person_df copy
P_copy['identifier'] = P_copy['identifier'].map(hp_uuid_mapping).fillna(P_copy['identifier'])

Update the dataset with variations from the golden_standard

In [56]:
# Dictionary to store all dataframes by entity type

# List of entity types and their corresponding variable names
entity_var_map = {
    'Address': 'A_copy',
    'ContactPoint': 'CP_copy',
    'Person': 'P_copy',
    'HealthcareOrganization': 'HO_copy',
    'ServiceDepartment': 'SD_copy',
    'HealthcarePersonnel': 'HP_copy'
}

# Build the entity_dataframes dictionary dynamically
entity_dataframes_copy = {}
for entity_type, var_name in entity_var_map.items():
    if var_name in locals():
        entity_dataframes_copy[entity_type] = locals()[var_name]
    else:
        print(f"Warning: Variable {var_name} not found for entity type {entity_type}")

# Iterate through each row in the golden record
for _, row in golden_record.iterrows():
    entity_type = row['entity_type']
    
    # Check if we have a dataframe for this entity type
    if entity_type in entity_dataframes_copy:
        df = entity_dataframes_copy[entity_type]
        
        # Find the matching row using original_id
        mask = df['anchor'] == row['original_id']
        
        
        # If a match is found, update the specified field with the varied value
        if mask.any():
            field_name = row['field_name']
            if field_name in df.columns:
                df.loc[mask, field_name] = row['varied_value']  
            else:
                print(f"Warning: Field {field_name} not found in {entity_type} dataframe")
        else:
            print(f"Warning: No row with identifier {row['original_id']} found in {entity_type} dataframe")
    else:
        print(f"Warning: No dataframe found for entity type {entity_type}")

Save the dataframes 

In [58]:
import os


# # Save each dataframe to CSV in the new directory
# entity_dataframes_copy['Address'].to_csv('Data_source/Sample_35_train/train_data_dupe/Address.csv', index=False)
# entity_dataframes_copy['ContactPoint'].to_csv('Data_source/Sample_35_train/train_data_dupe/ContactPoint.csv', index=False)
# entity_dataframes_copy['HealthcareOrganization'].to_csv('Data_source/Sample_35_train/train_data_dupe/HealthcareOrganization.csv', index=False)
# entity_dataframes_copy['HealthcarePersonnel'].to_csv('Data_source/Sample_35_train/train_data_dupe/HealthcarePersonnel.csv', index=False)
# entity_dataframes_copy['Person'].to_csv('Data_source/Sample_35_train/train_data_dupe/Person.csv', index=False)
# entity_dataframes_copy['ServiceDepartment'].to_csv('Data_source/Sample_35_train/train_data_dupe/ServiceDepartment.csv', index=False)

noise = 'high'

entity_dataframes_copy['Address'].to_csv(f'Data_source/Sample_15_test/sample_dup/Address_{noise}.csv', index=False)
entity_dataframes_copy['ContactPoint'].to_csv(f'Data_source/Sample_15_test/sample_dup/ContactPoint_{noise}.csv', index=False)
entity_dataframes_copy['HealthcareOrganization'].to_csv(f'Data_source/Sample_15_test/sample_dup/HealthcareOrganization_{noise}.csv', index=False)
entity_dataframes_copy['HealthcarePersonnel'].to_csv(f'Data_source/Sample_15_test/sample_dup/HealthcarePersonnel_{noise}.csv', index=False)
entity_dataframes_copy['Person'].to_csv(f'Data_source/Sample_15_test/sample_dup/Person_{noise}.csv', index=False)
entity_dataframes_copy['ServiceDepartment'].to_csv(f'Data_source/Sample_15_test/sample_dup/ServiceDepartment_{noise}.csv', index=False)

In [59]:
print(golden_record[['entity_type', 'original_id']].head())
for entity_name, df in entity_dataframes.items():
    print(entity_name, df[['anchor', 'identifier']].head())

  entity_type                           original_id
0     Address  b2b5591a-9d7f-442b-85e2-b0787a3e9905
1     Address  d693bf11-7721-4931-9734-2123e3ed9637
2     Address  fbfa7549-13ee-4ee8-950e-18400e60f583
3     Address  1c7bc6c8-d62f-4e8c-abca-09663eef04eb
4     Address  406ec398-9a4a-4a9c-a735-a17352ec0382
Address                                  anchor                            identifier
0  daf66c5f-2577-4ffa-887a-746319c16a0d  daf66c5f-2577-4ffa-887a-746319c16a0d
1  19d5f970-98b3-4c6e-8a14-b90a7795e986  19d5f970-98b3-4c6e-8a14-b90a7795e986
2  bb0378eb-7a62-422e-9d69-d9fc4b1cb8bd  bb0378eb-7a62-422e-9d69-d9fc4b1cb8bd
3  ef7e85ec-a417-456f-a9ee-7f3d0ff030b8  ef7e85ec-a417-456f-a9ee-7f3d0ff030b8
4  7d859725-c707-4ef9-86c3-744cc88e03b6  7d859725-c707-4ef9-86c3-744cc88e03b6
ContactPoint                                  anchor                            identifier
0  12f175ff-ae3b-46ec-9a27-d85888c132ad  12f175ff-ae3b-46ec-9a27-d85888c132ad
1  eb21a3f6-e6fd-48e8-969c-91c278601602  eb

update duplicate id's 


In [ ]:
import pandas as pd


Address_df = pd.read_csv('Data_source/Sample_15_test/sample_dup/Address_low.csv')  
ContactPoint_df = pd.read_csv('Data_source/Sample_15_test/sample_dup/ContactPoint_low.csv')
HealthcareOrganization_df = pd.read_csv('Data_source/Sample_15_test/sample_dup/HealthcareOrganization_low.csv')
HealthcarePersonnel_df = pd.read_csv('Data_source/Sample_15_test/sample_dup/HealthcarePersonnel_low.csv')
Person_df = pd.read_csv('Data_source/Sample_15_test/sample_dup/Person_low.csv')
ServiceDepartment_df = pd.read_csv('Data_source/Sample_15_test/sample_dup/ServiceDepartment_low.csv')

entity_dataframes_low = {
    'Address': Address_df,
    'ContactPoint': ContactPoint_df,
    'HealthcareOrganization': HealthcareOrganization_df,
    'HealthcarePersonnel': HealthcarePersonnel_df,
    'Person': Person_df,
    'ServiceDepartment': ServiceDepartment_df
}

Reupdate the golden standard to have matching duplicate IDs

In [ ]:
# Update the duplicate_id in the golden_record using anchor matching
golden_record = pd.read_csv('test_golden_standard_low.csv', encoding='latin1')
for entity_name, df in entity_dataframes_low.items():
    # Ensure 'anchor' and 'identifier' columns exist
    if 'anchor' in df.columns and 'identifier' in df.columns:
        # Create a mapping from anchor to identifier
        anchor_to_identifier = dict(zip(df['anchor'], df['identifier']))
        # Update duplicate_id in golden_record where entity_type matches
        mask = golden_record['entity_type'] == entity_name
        golden_record.loc[mask, 'duplicate_id'] = golden_record[mask]['original_id'].map(anchor_to_identifier).fillna(golden_record[mask]['duplicate_id'])

# Save or display the updated golden_record as needed
golden_record.to_csv('test_golden_standard_low_updated.csv', index=False, encoding='utf-8')
golden_record.head()

NameError: name 'golden_record' is not defined